In [1]:
! hdfs dfs -rm /user/cloudera/train.csv
! tail -n +2 ./small3.train.csv | hdfs dfs -put - /user/cloudera/train.csv
! hdfs dfs -ls -h

16/03/28 08:08:09 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/cloudera/train.csv
Found 4 items
drwxr-xr-x   - cloudera cloudera          0 2016-03-26 09:01 kv1.parquet
drwxr-xr-x   - cloudera cloudera          0 2016-03-26 08:56 people.parquet
-rw-r--r--   1 cloudera cloudera      1.5 M 2016-03-28 08:08 train.csv
drwxr-xr-x   - cloudera cloudera          0 2016-03-26 09:46 train_parquet


In [2]:
from pyspark.sql import HiveContext
hiveContext = HiveContext(sc)
print(hiveContext)
print(hiveContext.tableNames("default"))
print(hiveContext.sql("show databases").show())
hiveContext.sql("SET hive.metastore.warehouse.dir=hdfs:/user/hive/warehouse").show()

[]
+-------+
| result|
+-------+
|default|
+-------+

None
+--------------------+--------------------+
|                 key|               value|
+--------------------+--------------------+
|hive.metastore.wa...|hdfs:/user/hive/w...|
+--------------------+--------------------+



In [3]:
for i in [
    """drop table train_csv""",
    """create table train_csv (
id string, click int, hour string, C1 string, banner_pos string, site_id string, site_domain string, site_category string, app_id string, app_domain string, app_category string, device_id string, device_ip string, device_model string, device_type string, device_conn_type string, C14 string, C15 string, C16 string, C17 string, C18 string, C19 string, C20 string, C21 string
)
row format delimited fields terminated by ','
""",
          """
load data inpath 'hdfs:/user/cloudera/train.csv'
into table train_csv
   """,
          """
select * from train_csv limit 1
"""]:
    print(i)
#    ! echo "{i}" + ";" | hive
    print(hiveContext.sql(i).show())

drop table train_csv
++
||
++
++

None
create table train_csv (
id string, click int, hour string, C1 string, banner_pos string, site_id string, site_domain string, site_category string, app_id string, app_domain string, app_category string, device_id string, device_ip string, device_model string, device_type string, device_conn_type string, C14 string, C15 string, C16 string, C17 string, C18 string, C19 string, C20 string, C21 string
)
row format delimited fields terminated by ','

+------+
|result|
+------+
+------+

None

load data inpath 'hdfs:/user/cloudera/train.csv'
into table train_csv
   
+------+
|result|
+------+
+------+

None

select * from train_csv limit 1

+-------------------+-----+--------+----+----------+--------+-----------+-------------+--------+----------+------------+---------+---------+------------+-----------+----------------+-----+---+---+----+---+---+---+---+
|                 id|click|    hour|  c1|banner_pos| site_id|site_domain|site_category|  app_id|app_d

In [4]:
df = hiveContext.table("train_csv")
%time df.groupBy("click").count().show()

+-----+-----+
|click|count|
+-----+-----+
|    0|16586|
|    1| 3412|
+-----+-----+

CPU times: user 2 ms, sys: 2 ms, total: 4 ms
Wall time: 1.94 s


In [5]:
%time df.groupBy("banner_pos").count().show()

+----------+-----+
|banner_pos|count|
+----------+-----+
|         0|16108|
|         1| 3882|
|         4|    4|
|         5|    4|
+----------+-----+

CPU times: user 4 ms, sys: 3 ms, total: 7 ms
Wall time: 980 ms
